# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846806044534                   -0.70    4.5         
  2   -7.852310762686       -2.26       -1.53    1.0   24.4ms
  3   -7.852615534800       -3.52       -2.55    1.5   26.3ms
  4   -7.852645952450       -4.52       -2.89    2.5   33.5ms
  5   -7.852646504089       -6.26       -3.18    1.2   25.9ms
  6   -7.852646678188       -6.76       -4.00    1.0   24.8ms
  7   -7.852646686274       -8.09       -5.01    1.8   29.6ms
  8   -7.852646686722       -9.35       -5.48    1.8   29.9ms
  9   -7.852646686729      -11.16       -6.15    1.5   27.3ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846906888062                   -0.70           4.8         
  2   -7.852530369091       -2.25       -1.64   0.80    2.0    253ms
  3   -7.852637315490       -3.97       -2.72   0.80    1.0   23.5ms
  4   -7.852646517647       -5.04       -3.29   0.80    2.2   30.4ms
  5   -7.852646681933       -6.78       -4.14   0.80    1.2   24.9ms
  6   -7.852646686577       -8.33       -4.84   0.80    1.8   28.2ms
  7   -7.852646686723       -9.84       -5.69   0.80    1.8   27.4ms
  8   -7.852646686730      -11.15       -6.55   0.80    2.2   29.9ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.413807e+01     3.235334e+00
 * time: 0.05756497383117676
     1     1.210684e+00     1.761483e+00
 * time: 0.2732560634613037
     2    -1.600983e+00     2.184312e+00
 * time: 0.2984941005706787
     3    -3.948929e+00     1.946626e+00
 * time: 0.3341829776763916
     4    -5.462302e+00     1.724917e+00
 * time: 0.37096095085144043
     5    -7.023186e+00     8.128526e-01
 * time: 0.40739893913269043
     6    -7.281335e+00     8.270270e-01
 * time: 0.43241000175476074
     7    -7.640726e+00     3.189053e-01
 * time: 0.45818305015563965
     8    -7.726388e+00     1.459047e-01
 * time: 0.48302793502807617
     9    -7.765431e+00     1.088429e-01
 * time: 0.507519006729126
    10    -7.801351e+00     9.988836e-02
 * time: 0.5320699214935303
    11    -7.826567e+00     7.563117e-02
 * time: 0.5575640201568604
    12    -7.846316e+00     4.114880e-02
 * time: 0.5826830863952637
    13    -7.850531e+00     4.341475e-02
 * time: 0.60766

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846862207895                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645925433                   -1.64         
  2   -7.852646686730       -6.12       -3.71    1.95s
  3   -7.852646686730      -13.32       -7.26    186ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 6.213438675174651e-7
|ρ_newton - ρ_scfv| = 1.9935469726770303e-7
|ρ_newton - ρ_dm|   = 7.657485560488458e-10
